In [11]:
from src.causalchange.gen.generate import GenDataType, GSType, DagType
%load_ext autoreload
%autoreload 2

import os, sys
import networkx as nx
import numpy as np
import pandas as pd

from pathlib import Path
from src.causalchange.gen.generate import gen_data_type, FunType, NoiseType, IvType
from src.causalchange.causal_change import CausalChange

#sys.path.insert(0, os.path.abspath(".."))
# or sys.path.insert(0, os.path.abspath("/workspace"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [108]:

seed = 42


params = {
    'N': 6,
    'S': 500,
    'P': 0.4,
    'IVM': GenDataType.IID,
    'IVT': IvType.COEF,
    'GS': GSType.GRAPH,
    'DG': DagType.ERDOS,
    'F': FunType.LIN,
    'NS': NoiseType.GAUSS,
}

X, truths = gen_data_type(params, seed)
true_g = truths["true_g"]
Path("../demo/datasets/").mkdir(parents=True, exist_ok=True)
pd.DataFrame(X).to_csv(f'../demo/datasets/synthetic_IID.tsv',  sep='\t', index=False)

pd.DataFrame(nx.to_numpy_array(truths["true_g"])).to_csv('../demo/datasets/synthetic_IID_g.tsv',
                                                         sep='\t', index=False)


<function DataGen.<lambda> at 0x0000026F606F0790>


In [105]:
from src.causalchange.cc_types import DataMode, ScoreType, GraphSearch
from src.causalchange.causal_change import CausalChange

cc = CausalChange(data_mode=DataMode.IID, score_type=ScoreType.GP.value.EXACT, graph_search=GraphSearch.TOPIC, vb=3,
                  truths=truths)
dag = cc.fit(X)

DataMode.IID GPType.EXACT 
	*** CausalChange (gp) ***


KeyboardInterrupt: 

In [21]:
print(dag.edges)
print(true_g.edges)

[(0, 2), (0, 5), (3, 2), (3, 4), (3, 5)]
[(0, 2), (0, 5), (3, 2), (3, 4), (3, 5)]


In [109]:
from src.causalchange.cc_types import  score_type_get_all

for score in score_type_get_all():
    cc = CausalChange(data_mode=DataMode.IID, score_type=score, graph_search=GraphSearch.TOPIC, vb=1,
                  truths=truths)
    dag = cc.fit(X)
    print(dag.edges)
    print(true_g.edges)

DataMode.IID ScoreType.LIN 
	*** CausalChange (lin) ***
[(0, 5), (1, 2)]
[(0, 1), (0, 5), (3, 1), (4, 1), (5, 1)]
DataMode.IID ScoreType.GAM 
	*** CausalChange (gam) ***
[(1, 2), (1, 4), (5, 0)]
[(0, 1), (0, 5), (3, 1), (4, 1), (5, 1)]
DataMode.IID ScoreType.SPLINE 
	*** CausalChange (spline) ***
[(4, 2)]
[(0, 1), (0, 5), (3, 1), (4, 1), (5, 1)]
DataMode.IID GPType.EXACT 
	*** CausalChange (gp) ***
[(5, 0)]
[(0, 1), (0, 5), (3, 1), (4, 1), (5, 1)]
DataMode.IID GPType.FOURIER 
	*** CausalChange (ff) ***
[(0, 5), (1, 2)]
[(0, 1), (0, 5), (3, 1), (4, 1), (5, 1)]


In [111]:
for score in score_type_get_all():
    cc = CausalChange(data_mode=DataMode.IID, score_type=score, graph_search=GraphSearch.GLOBE, vb=2,
                  truths=truths)
    dag = cc.fit(X)
    print(dag.edges)
    print(true_g.edges)

DataMode.IID ScoreType.LIN 
	*** CausalChange (lin) ***
DAG search result:5->0, 2->1
[(2, 1), (5, 0)]
[(0, 1), (0, 5), (3, 1), (4, 1), (5, 1)]
DataMode.IID ScoreType.GAM 
	*** CausalChange (gam) ***
DAG search result:0->5, 1->2
[(0, 5), (1, 2)]
[(0, 1), (0, 5), (3, 1), (4, 1), (5, 1)]
DataMode.IID ScoreType.SPLINE 
	*** CausalChange (spline) ***
DAG search result:0->2, 4->5, 5->0, 5->3
[(0, 2), (4, 5), (5, 0), (5, 3)]
[(0, 1), (0, 5), (3, 1), (4, 1), (5, 1)]
DataMode.IID GPType.EXACT 
	*** CausalChange (gp) ***
DAG search result:5->0, 2->1
[(2, 1), (5, 0)]
[(0, 1), (0, 5), (3, 1), (4, 1), (5, 1)]
DataMode.IID GPType.FOURIER 
	*** CausalChange (ff) ***
DAG search result:5->0, 1->2
[(1, 2), (5, 0)]
[(0, 1), (0, 5), (3, 1), (4, 1), (5, 1)]


In [145]:

seed = 42

n_c = 10
params = {
    'N': 6,
    'S': 500,
    'P': 0.4,
    'C': n_c,
    'PC': 1,
    'K': 4,
    'IVM': GenDataType.MULTI_CONTEXT,
    'IVT': IvType.COEF,
    'GS': GSType.GRAPH,
    'DG': DagType.ERDOS,
    'F': FunType.LIN,
    'NS': NoiseType.GAUSS,
}

X, truths = gen_data_type(params, seed)
true_g = truths["true_g"]
Path("../demo/datasets/").mkdir(parents=True, exist_ok=True)
for c_i in range(len(X)):
     pd.DataFrame(X[c_i]).to_csv(f'../demo/datasets/synthetic_IID.tsv',  sep='\t', index=False)

pd.DataFrame(nx.to_numpy_array(truths["true_g"])).to_csv('../demo/datasets/synthetic_IID_g.tsv',
                                                         sep='\t', index=False)


In [149]:

cc = CausalChange(data_mode=DataMode.CONTEXTS, score_type=ScoreType.GP.value.FOURIER, graph_search=GraphSearch.TOPIC, vb=4,
                  truths=truths)
dag = cc.fit(X)

DataMode.CONTEXTS GPType.FOURIER 
	*** CausalChange, multi-context setting (ff) ***
	Scoring spurious edge [0] -> 1	1020.09
	Scoring spurious edge [0] -> 2	1024.14


KeyboardInterrupt: 

In [101]:
print(dag.edges)
print(true_g.edges)

[(1, 0), (1, 5), (4, 0), (4, 2), (4, 5), (5, 2)]
[(1, 0), (1, 5), (2, 5), (3, 5), (4, 0), (4, 2)]


In [147]:

cc = CausalChange(data_mode=DataMode.CONTEXTS, score_type=ScoreType.CI.value.KCI, graph_search=GraphSearch.TOPIC, vb=4,
                  truths=truths)
dag = cc.fit(X)

DataMode.CONTEXTS CIType.KCI 
	*** CausalChange, multi-context setting (kci) ***
	Scoring spurious edge [0] -> 1	1.36
	Scoring spurious edge [0] -> 2	-0.00
	Scoring spurious edge [0] -> 3	0.47
	Scoring spurious edge [0] -> 4	1.36
	Scoring spurious edge [0] -> 5	-0.00
	Scoring spurious edge [1] -> 0	-0.00
	Scoring rev edge [1] -> 2	-0.00
	Scoring spurious edge [1] -> 3	-0.00
	Scoring spurious edge [1] -> 4	1.85
	Scoring spurious edge [1] -> 5	-0.00
	Scoring spurious edge [2] -> 0	-0.00
	Scoring causal edge [2] -> 1	1.37
	Scoring spurious edge [2] -> 3	-0.00
	Scoring causal edge [2] -> 4	1.36
	Scoring spurious edge [2] -> 5	-0.00
	Scoring spurious edge [3] -> 0	-0.00
	Scoring spurious edge [3] -> 1	1.36
	Scoring spurious edge [3] -> 2	-0.00
	Scoring causal edge [3] -> 4	1.36
	Scoring spurious edge [3] -> 5	-0.00
	Scoring spurious edge [4] -> 0	-0.00
	Scoring spurious edge [4] -> 1	-0.00
	Scoring rev edge [4] -> 2	-0.00
	Scoring rev edge [4] -> 3	-0.00
	Scoring rev edge [4] -> 5	-0.00
	Sc

In [148]:
print(dag.edges)
print(true_g.edges)

[(0, 4), (0, 5), (4, 1), (4, 3)]
[(2, 1), (2, 4), (3, 4), (5, 4)]
